# Human in the Loop

Your durable research application now survives crashes and automatically retries failures. But there's a critical gap: **it runs completely autonomously**.

Imagine this scenario: Your AI generates research on "best travel spots for summer 2025", your application generates a PDF, and sends it to your client—all automatically.

Then your client calls: "I want cheaper options!"

Your application had no way to pause and ask for clarification or approval. It just executed straight through to completion.

**Real-world AI applications need human interaction**—to provide feedback, approve decisions, or clarify requirements. But adding human input to automated workflows introduces new challenges: What if the user's browser crashes while they're reviewing? What if they close the tab and come back later? How do you prevent losing expensive LLM work while waiting for approval?

In this section, we'll solve these problems by making your AI application interactive and durable.

### Most GenAI Apps Still Engage Humans

While some AI applications may operate entirely autonomously, many require human intervention. They may provide input on launch or at various points throughout the execution.

Examples:
  - Validation at critical decision points
  - Final review before implementation
  - Feedback loops

This can happen in a chain workflow like if the human wants to confirm the research before outputting it into a PDF:

<img src="https://images.ctfassets.net/0uuz8ydxyd9p/70SBemKQHnqfLxoHgPovQX/33f3a0b6cfc96eae2d17d1a463079560/Screenshot_2025-07-08_at_10.26.26%C3%A2__AM.png" />


Or there can be continuous feedback cycle in a human-in-the-loop system. The AI agent performs work based on input or previous feedback, then pauses to wait for human review. This feedback is fed back into the AI agent, which then continues execution based on the human's decision:

<img src="https://i.postimg.cc/43pm5mfw/hitl-loop.png" width="300"/>



### Example: Customer Support Agent with Human Oversight

The Scenario:
Customer contacts support: "My order hasn't arrived and I need it urgently"

**How Human Interaction Works:**

1. **Initial Input** - Customer provides problem; human sets constraints like maximum refund amounts or escalation thresholds

2. **AI Analysis** - LLM checks order status and finds "delayed in transit," then proposes: "Process a $75 expedited replacement"

3. **Human Decision Point** - System pauses for approval. Human can:
   - Approve the proposed action
   - Modify the approach ("Offer refund instead")
   - Add context ("VIP customer - upgrade to overnight")

**Why This Matters:**
- **Cost control** - Expensive decisions require human approval
- **Security** - Prevent unauthorized refunds or account changes
- **Risk mitigation** - Human judgment for edge cases and exceptions
- **Compliance** - Legal/regulatory requirements for certain actions

### Challenges in Non-Durable Human in the Loop Processes

While human interaction points are valuable for AI applications, implementing them reliably presents significant technical challenges. Without durable execution, human input can be lost during system failures, leading to unpredictable behavior.

**Consider the following scenario**

- A user needs to approve a transaction.
- As they are doing this, the website goes down.
- How do you mitigate this?
  - Do we notify the user to approve the payment again? (creating confusion since the user already 'approved')
  - Do we assume approval and risk processing an unauthorized payment?

Without durable processes, you're forced to choose between security, user experience, and reliability.

<img src="https://i.postimg.cc/C5G3rpBy/Screenshot-2025-10-08-at-9-19-32-AM.png" />

#### It's **distributed system challenges** all over again!

### But with **Durable Execution**, the Workflow LIVES Through These Interactions!

With Temporal's durable execution, the workflow instance **persists throughout the entire human interaction**:

**What this means in practice:**
- **User's approval is durably stored** - When a user clicks "approve", that decision is saved in the workflow history
- **No re-approval needed** - If the website crashes after approval, the workflow resumes with the approval already recorded
- **Automatic recovery** - System failures don't lose progress; the workflow picks up exactly where it left off
- **User can walk away** - Close the browser, shut down the laptop—the workflow continues running on the server

**The key insight:** Instead of managing complex coordination between services, queues, and databases to handle human input, you write straightforward code that waits for human decisions. Temporal handles all the reliability, state management, and recovery automatically.

<img src="https://i.postimg.cc/bYR8R1QC/hitl-durable-execution.png" width="400"/>

### Signals in Temporal

In Temporal, human interaction in Temporal systems is achieved through a [Temporal Signal](https://docs.temporal.io/develop/python/message-passing#signals).

A Signal is a:
* Message sent asynchronously to a running Workflow Execution
* Used to to change the state and control the flow of a Workflow Execution.

*Take 2 minutes to discuss with your neighbor when we might use a Signal, then we'll share answers*

### Example Signal Usage

1. The user initiates a workflow with an initial request
2. The workflow processes the request and determines what information or approval is needed
3. The workflow pauses and waits for user input via Signal, such as:
    - Additional information or clarification
    - Permission to proceed with an action
    - Selection between multiple options
4. The user sends a Signal with their response
5. The workflow resumes execution based on the Signal received
6. Steps repeat as needed until the workflow completes its task

### Durably Storing Human Interactions

Let's go back to the user approving a payment example now. With Temporal, when the user clicks "approve" in the finance portal, the approval decision gets durably stored.

The user can close the browser, go to lunch, and the Workflow will continue running in the background.

If the payment gateway times out, returns an error or becomes unavailable, Temporal automatically retries the payment step. IT does not need to re-ask the user for approval, because that decision is already durably stored in the Workflow state.

- **No duplicate work** (user does not have to re-approve the same expense)
- **No manual intervention** (does not need to manually reconcile failed payments or investigate whether an expense was actually approved)
- **Reliable processing** (business can count on approved expenses being paid)

### Let's Implement a Signal!

Let's update our application. After receiving a Signal, your Workflow's main execution logic must evaluate the Signal data and determine the appropriate response. This is where the Workflow's business logic intersects with human input.

1. We will now give the user the opportunity to either 
    - Accept the research results from the LLM, or
    - Refine the output with an additional prompt
2. So we'll loop

<img src="https://i.postimg.cc/BZPYWhHZ/signal-goal.png" width="200"/>

### Demo (Expand for instructor notes or to run on your own)
<!--
1. Clone this repository: `https://github.com/temporalio/edu-ai-workshop-agentic-loop`. The instructions will also be in the README.
2. From the `src/module_one_03_human_in_the_loop` directory, run `app.py` with `python app.py`.
3. In one terminal window, run your Worker with `python worker.py`.
4. In another terminal window, execute your Workflow with `python starter.py`.
5. You'll be prompted to enter a research topic or question in the CLI.
6. Once you do, you'll be prompted with the ability to Signal or Query the Workflow.
7. Type 'query' and you'll see the output in the terminal window where you started your Workflow Execution.
8. Time to demonstrate Signals. Back in the terminal window when you started your Workflow Execution, you'll see that you are prompted to choose one of the two options:
    a. Approve of this research and if you would like it to create a PDF (type 'keep' to send a Signal to the Workflow to create the PDF).
    b. Modify the research by adding extra info to the prompt (type 'edit' to modify the prompt and send another Signal to the Workflow to prompt the LLM again).
9. Demonstrate the modification by typing `edit`.
10. Enter additional instructions (e.g.: "turn this into a poem") and see the new output in the terminal window by typing `query` again.
11. Finally, show that you can keep changing the execution path of your Workflow Execution by typing `keep`. Show that the PDF has appeared in your `module_one_03_human_in_the_loop` directory.
-->

### Setup Notebook

Run the following code blocks to install various packages and tools necessary to run this notebook

**Be sure to add your .env file again. It doesn't persist across notebooks or sesions**

In [ ]:
# We'll first install the necessary packages for this workshop.

%pip install --quiet temporalio litellm reportlab python-dotenv

In [ ]:
# Create .env file
with open(".env", "w") as fh:
  fh.write("LLM_API_KEY = YOUR_API_KEY\nLLM_MODEL = openai/gpt-4o")

  # Now open the file and replace YOUR_API_KEY with your API key.

In [ ]:
# Load environment variables and configure LLM settings
import os
from dotenv import load_dotenv

load_dotenv(override=True)

# Get LLM_API_KEY environment variable and print it to make sure that your .env file is properly loaded.
LLM_MODEL = os.getenv("LLM_MODEL", "openai/gpt-4o")
LLM_API_KEY = os.getenv("LLM_API_KEY", None)
print("LLM API Key", LLM_API_KEY)

In [ ]:
# This allows us to run the Temporal Asyncio event loop within the event loop of Jupyter Notebooks
import nest_asyncio
nest_asyncio.apply()

In [ ]:
 # Running this will download the Temporal CLI

!curl -sSf https://temporal.download/cli.sh | sh

### Make Sure Your Temporal Web UI is Running

1. You should have the Temporal Server running in your terminal (run `temporal server start-dev` if not).
2. Then in your `Ports` tab on the bottom of this screen, find `8233` and click on the Globe icon to open the Temporal Web UI.

In [ ]:
# Run these code blocks to load our dataclasses and Activities (from the previous workshop) into the program
from dataclasses import dataclass
from temporalio import activity
from litellm import completion, ModelResponse
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle

# Dataclasses

@dataclass
class LLMCallInput:
  prompt: str
  llm_api_key: str
  llm_model: str

@dataclass
class PDFGenerationInput:
  content: str
  filename: str = "research_pdf.pdf"

@dataclass
class GenerateReportInput:
    prompt: str
    llm_api_key: str
    llm_research_model: str = "openai/gpt-4o"
    llm_image_model: str = "dall-e-3"

@dataclass
class GenerateReportOutput:
    result: str

@activity.defn
def llm_call(input: LLMCallInput) -> ModelResponse:
    response = completion(
      model=input.llm_model,
      api_key=input.llm_api_key,
      messages=[{ "content": input.prompt,"role": "user"}]
    )
    return response

# Activities

@activity.defn
def create_pdf(input: PDFGenerationInput) -> str:
    print("Creating PDF document...")

    doc = SimpleDocTemplate(input.filename, pagesize=letter)
    styles = getSampleStyleSheet()
    title_style = ParagraphStyle(
        'CustomTitle',
        parent=styles['Heading1'],
        fontSize=24,
        spaceAfter=30,
        alignment=1
    )

    story = []
    title = Paragraph("Research Report", title_style)
    story.append(title)
    story.append(Spacer(1, 20))
    paragraphs = input.content.split('\n\n')
    for para in paragraphs:
        if para.strip():
          p = Paragraph(para.strip(), styles['Normal'])
          story.append(p)
          story.append(Spacer(1, 12))

    doc.build(story)

    print(f"SUCCESS! PDF created: {input.filename}")
    return input.filename

## Let's Create Our Loop

After generating the research, and before creating a PDF report, the user can review the research. The user has two choices to send to the Workflow:

- Keep the research and generate a PDF report by signaling "keep"
- Modify the research before generating a PDF report by signaling "edit"

In [ ]:
# Step 1: Set the condition to check if the user decision is `UserDecision.EDIT`

from temporalio import workflow
from datetime import timedelta

# sandboxed=False is a Notebook only requirement. You normally don't do this
@workflow.defn(sandboxed=False)
class GenerateReportWorkflow:
    def __init__(self) -> None:
        self._current_prompt: str = ""

    @workflow.run
    async def run(self, input: GenerateReportInput) -> GenerateReportOutput:
        self._current_prompt = input.prompt

    llm_call_input = LLMCallInput(
        prompt=self._current_prompt,
        llm_api_key=input.llm_api_key,
        llm_model=input.llm_research_model,
    )
    
    # Continue looping until the user approves the research
    continue_agent_loop = True

    # Execute the LLM call to generate research based on the current prompt
    while continue_agent_loop:
        research_facts = await workflow.execute_activity(
            llm_call,
            llm_call_input,
            start_to_close_timeout=timedelta(seconds=30),
        )

        # User approved the research - exit the loop and proceed to PDF generation
        if self._user_decision.decision == UserDecision.KEEP:
            workflow.logger.info("User approved the research. Creating PDF...")
            continue_agent_loop = False
        # User wants to edit the research - update the prompt and loop again
        elif self._user_decision.decision == "": # TODO: Set the condition to check if the user decision is `EDIT`
            workflow.logger.info("User requested research modification.")
            if self._user_decision.additional_prompt != "":
                # Append the user's additional instructions to the existing prompt
                self._current_prompt = (
                    f"{self._current_prompt}\n\nAdditional instructions: {self._user_decision.additional_prompt}"
                )
            else:
                workflow.logger.info("No additional instructions provided. Regenerating with original prompt.")
            # Update the Activity input with the modified prompt for the next iteration
            llm_call_input.prompt = self._current_prompt

In [ ]:
# Run this cell to load and display the solution
from pathlib import Path
from IPython.display import display, Markdown
import os

notebook_dir = Path(os.getcwd())
solution_file = notebook_dir / "Solutions_03_Human_in_the_Loop" / "loop_solution.py"

code = solution_file.read_text()

print("Solution loaded:")
display(Markdown(f"```python\n{code}\n```"))

### Let's Implement a Signal!

We have now added the Workflow main logic that checks the state and reacts accordingly! Now, we need to do three things:

1. **Define the structure for the data being supplied by the user** 
2. **Implement the point(s) in the flow where user input is sought**          
3. **Implement a Signal handler that is called when the user supplies the information.**
    - Implement a Signal handler method decorated with `@workflow signal`
    - This signal handler will write into the above defined data structure

Let's first look at storing the Signal state.

### Step 1 - Create a Model

- Create a model for the Signal to be stored in
- Similar to Activities and Workflows, `dataclasses` are recommended here

In [ ]:
# This defines a string enum and a dataclass for handling user decisions in a Temporal workflow.
# We will send a `UserDecision` as a Signal to our Research Workflow letting the Workflow know if 
# we want to keep or edit the research or if we want to wait for further decision

# Step 1: Set the additional prompt to be a string that default as an empty string
# Step 2: Run this code block
from dataclasses import dataclass
from enum import StrEnum

class UserDecision(StrEnum):
    KEEP = "KEEP"
    EDIT = "EDIT"
    WAIT = "WAIT"
    
@dataclass
class UserDecisionSignal: # A data structure to send user decisions via Temporal Signals
    decision: UserDecision
    additional_prompt: # TODO Set this to be a string that defaults as an empty string

In [ ]:
# Run this cell to load and display the solution
from pathlib import Path
from IPython.display import display, Markdown
import os

notebook_dir = Path(os.getcwd())
solution_file = notebook_dir / "Solutions_03_Human_in_the_Loop" / "model_solution.py"

code = solution_file.read_text()

print("Solution loaded:")
display(Markdown(f"```python\n{code}\n```"))

### Step 2 - Storing the Signal State

Remember, the workflow needs a place to remember what Signals it has received.

- Use instance variables to persist signal data across Workflow Execution
- Can be a simple variable, or a Queue for handling many Signals
- Initialize with default values that indicates "no Signal received yet". In the example above, `WAIT` is the default state.

In [ ]:
# Step 1: Set the instance variable of the user decision to default to WAIT
from datetime import timedelta
from temporalio import workflow

# sandboxed=False is a Notebook only requirement. You normally don't do this
@workflow.defn(sandboxed=False)
class GenerateReportWorkflow:
    def __init__(self) -> None:
        self._current_prompt: str = ""
        # Instance variable to store Signal data
        self._user_decision: UserDecisionSignal = UserDecisionSignal(decision=UserDecision.____) # TODO Set the default state of the user decision to be WAIT

    @workflow.run
    async def run(self, input: GenerateReportInput) -> GenerateReportOutput:
        self._current_prompt = input.prompt

        llm_call_input = LLMCallInput(
            prompt=self._current_prompt,
            llm_api_key=input.llm_api_key,
            llm_model=input.llm_research_model,
        )

        # Continue looping until the user approves the research
        continue_agent_loop = True

        # Execute the LLM call to generate research based on the current prompt
        while continue_agent_loop:
            research_facts = await workflow.execute_activity(
                llm_call,
                llm_call_input,
                start_to_close_timeout=timedelta(seconds=30),
            )

            # User approved the research - exit the loop and proceed to PDF generation
            if self._user_decision.decision == UserDecision.KEEP:
                workflow.logger.info("User approved the research. Creating PDF...")
                continue_agent_loop = False
            # User wants to edit the research - update the prompt and loop again
            elif self._user_decision.decision == UserDecision.EDIT:
                workflow.logger.info("User requested research modification.")
                if self._user_decision.additional_prompt != "":
                    # Append the user's additional instructions to the existing prompt
                    self._current_prompt = (
                        f"{self._current_prompt}\n\nAdditional instructions: {self._user_decision.additional_prompt}"
                    )
                else:
                    workflow.logger.info("No additional instructions provided. Regenerating with original prompt.")
                # Update the Activity input with the modified prompt for the next iteration
                llm_call_input.prompt = self._current_prompt

        pdf_generation_input = PDFGenerationInput(content=research_facts["choices"][0]["message"]["content"])

        pdf_filename: str = await workflow.execute_activity(
            create_pdf,
            pdf_generation_input,
            start_to_close_timeout=timedelta(seconds=20),
        )

        return GenerateReportOutput(result=f"Successfully created research report PDF: {pdf_filename}")

In [ ]:
# Run this cell to load and display the solution
from pathlib import Path
from IPython.display import display, Markdown
import os

notebook_dir = Path(os.getcwd())
solution_file = notebook_dir / "Solutions_03_Human_in_the_Loop" / "store_signal_solution.py"

code = solution_file.read_text()

print("Solution loaded:")
display(Markdown(f"```python\n{code}\n```"))

### Defining a Signal Handler
- A Signal is defined in your code and handled in your Workflow Definition
- A given Workflow Definition can support multiple Signals
- To define a Signal, set the Signal decorator `@workflow.signal` on the Signal function inside your Workflow class
- Signal methods define what happens when the Signal is received

In this example, when a client sends a Signal, this handler receives the UserDecisionSignal data and updates the `self._user_decision` in the Workflow.

In [ ]:
# Step 1: Add the `@workflow.signal` decorator above the `user_decision_signal` method
# Step 2: Update the instance variable `self._user_decision` by setting it to `decision_data`
# Reflection Question: What's still missing for this Signal implementation to work properly?
from temporalio import workflow

@workflow.defn(sandboxed=False) # sandboxed=False is a Notebook only requirement. You normally don't do this)
class GenerateReportWorkflow:
    def __init__(self) -> None:
        self._current_prompt: str = ""
        self._user_decision: UserDecisionSignal = UserDecisionSignal(decision=UserDecision.WAIT) # UserDecision Signal starts with WAIT as the default state

    # TODO Define the Signal handler with @workflow.signal
    async def user_decision_signal(self, decision_data: UserDecisionSignal) -> None:
        self._user_decision = # Update instance variable when Signal is received to decision_data

    @workflow.run
    async def run(self, input: GenerateReportInput) -> GenerateReportOutput:
      self._current_prompt = input.prompt

      llm_call_input = LLMCallInput(
          prompt=self._current_prompt,
          llm_api_key=input.llm_api_key,
          llm_model=input.llm_research_model,
      )
      # rest of code here

In [ ]:
# Run this cell to load and display the solution
from pathlib import Path
from IPython.display import display, Markdown
import os

notebook_dir = Path(os.getcwd())
solution_file = notebook_dir / "Solutions_03_Human_in_the_Loop" / "signal_handler_solution.py"

code = solution_file.read_text()

print("Solution loaded:")
display(Markdown(f"```python\n{code}\n```"))

# Waiting for a Signal
Now that we have a Signal handler to receive data, we need a way for the Workflow to pause and wait for that Signal to arrive. This is where workflow.wait_condition() comes in.

- Use workflow.wait_condition() to pause until Signal is received (user decides the next step)
- Creates a blocking checkpoint where the Workflow stops and waits
- Resumes execution only when specified condition becomes true
- Optionally accepts a timeout parameter: `workflow.wait_condition(lambda: condition, timeout=timedelta(hours=24))` - waits until Signal received OR timeout elapsed, whichever happens first

**Benefits**:
- **Resource efficiency**: During waiting periods, the Workflow instance consumes no CPU or memory. The Worker only "wakes up" when a Signal is received or when replaying history.
- **Durability**: If systems crash while waiting, the Workflow resumes exactly where it left off when systems recover

In [ ]:
# Step 1: Within the `await workflow.wait_condition(lambda: )` code, 
# set the lambda to when `self._user_decision.decision` is not set to `UserDecision.WAIT`
# Step 2: Reset the Signal state back to WAIT so we can receive the next user decision
# Step 3: Run this code block
from temporalio import workflow

@workflow.defn(sandboxed=False)
class GenerateReportWorkflow:
    def __init__(self) -> None:
        self._current_prompt: str = ""
        # Instance variable to store the Signal in
        self._user_decision: UserDecisionSignal = UserDecisionSignal(decision=UserDecision.WAIT)

    # Method to handle the Signal
    @workflow.signal
    async def user_decision_signal(self, decision_data: UserDecisionSignal) -> None:
        # Update the instance variable with the received Signal data
        self._user_decision = decision_data

    @workflow.run
    async def run(self, input: GenerateReportInput) -> GenerateReportOutput:
        self._current_prompt = input.prompt

        llm_call_input = LLMCallInput(
            prompt=self._current_prompt,
            llm_api_key=input.llm_api_key,
            llm_model=input.llm_research_model,
        )

        continue_agent_loop = True

        while continue_agent_loop:
            research_facts = await workflow.execute_activity(
                llm_call,
                llm_call_input,
                start_to_close_timeout=timedelta(seconds=30),
            )

            self._research_result = research_facts["choices"][0]["message"]["content"]

            # Waiting for Signal with user decision
            await workflow.wait_condition(lambda: ) # TODO set the lambda to when `self._user_decision.decision` is not set to `UserDecision.WAIT`

            if self._user_decision.decision == UserDecision.KEEP:
                workflow.logger.info("User approved the research. Creating PDF...")
                continue_agent_loop = False
            elif self._user_decision.decision == UserDecision.EDIT:
                workflow.logger.info("User requested research modification.")
                if self._user_decision.additional_prompt != "":
                    self._current_prompt = (
                        f"{self._current_prompt}\n\nAdditional instructions: {self._user_decision.additional_prompt}"
                    )
                else:
                    workflow.logger.info("No additional instructions provided. Regenerating with original prompt.")
                llm_call_input.prompt = self._current_prompt

                # TODO Reset the Signal state back to WAIT so we can receive the next user decision

        pdf_generation_input = PDFGenerationInput(content=research_facts["choices"][0]["message"]["content"])

        pdf_filename: str = await workflow.execute_activity(
            create_pdf,
            pdf_generation_input,
            start_to_close_timeout=timedelta(seconds=20),
        )

        return GenerateReportOutput(result=f"Successfully created research report PDF: {pdf_filename}")

In [ ]:
# Run this cell to load and display the solution
from pathlib import Path
from IPython.display import display, Markdown
import os

notebook_dir = Path(os.getcwd())
solution_file = notebook_dir / "Solutions_03_Human_in_the_Loop" / "wait_condition_solution.py"

code = solution_file.read_text()

print("Solution loaded:")
display(Markdown(f"```python\n{code}\n```"))

## Sending Signals

- We can now run the Workflow and send our Signal
- There are multiple ways to send a Signal
  - Using a Temporal Client in an SDK
  - Using the Web UI
  - Using the `temporal` cli
- For this example, we will use a Temporal Client.

### Let's Test our Signal!

Now that your Signal is implemented, let's run the Worker and start the Workflow.

In [ ]:
# Let's set up our Worker. Run this code block to load it into the program
import concurrent.futures
from temporalio.client import Client
from temporalio.worker import Worker

async def run_worker() -> None:
    # Create client connected to server at the given address
    client = await Client.connect("localhost:7233", namespace="default")

    # Run the Worker
    with concurrent.futures.ThreadPoolExecutor(max_workers=100) as activity_executor:
        worker = Worker(
            client,
            task_queue="research",
            workflows=[GenerateReportWorkflow],
            activities=[llm_call, create_pdf],
            activity_executor=activity_executor
        )
        print(f"Starting the worker..")
        await worker.run()

print("Worker loaded!")

In [ ]:
# Run this to run the Worker
import asyncio
worker = asyncio.create_task(run_worker())

In [ ]:
# Start the Workflow! Run this codeblock
import uuid
from temporalio.client import Client

client = await Client.connect("localhost:7233", namespace="default")

print("Welcome to the Research Report Generator!")
prompt = input("Enter your research topic or question: ").strip()

if not prompt:
    prompt = "Give me 5 fun and fascinating facts about tardigrades. Make them interesting and educational!"
    print(f"No prompt entered. Using default: {prompt}")

# Asynchronous start of a Workflow
handle = await client.start_workflow(
    GenerateReportWorkflow.run,
    GenerateReportInput(prompt=prompt, llm_api_key=LLM_API_KEY),
    id=f"generate-research-report-workflow-{uuid.uuid4()}",
    task_queue="research",
)

print(f"Started workflow. Workflow ID: {handle.id}, RunID {handle.result_run_id}")

## Observing Signals in the Web UI

Refresh your Web UI. Look for the `Workflow Execution Signaled` Event. What was the input?

In [ ]:
# Stop the current Worker so we can start fresh with Query support
x = worker.cancel()

if x:
  print("Worker killed!")
else:
  print("Worker was not running. Nothing to kill")

## Part 2: Adding Query Support in Temporal

Now let's add **[Query](https://docs.temporal.io/develop/python/message-passing#queries)** support to our Workflow. Queries:

- Extract state to show the user
- Can be done during or even after the Workflow Execution has completed
    - In either case, there must be at least one running Worker for the Task Queue to which that Workflow belongs.
- Are a synchronous operation that that retrieve state from a Workflow Execution. Once a Query is issued, the Client waits for a response from the Workflow. 

Examples:
- **Monitor Progress of Long-Running Workflows**: A Client might want to receive updates on the progress, like the percentage of the task completed.
- **Retrieve Results**: Queries can be used to fetch the results of Activities without waiting for the entire Workflow to complete.

## Define and Handle a Query

Let's create a Query which will allow external clients to read the current research content from a running Workflow without interrupting its execution.

You can handle Queries by annotating a function within your Workflow with `@workflow.query`:

In [ ]:
# Adding a Query into our Workflow code
# Step 1: Initialize the _research_result to be an empty string
# Step 2: In your Query handler, returns the `_research_result` field
import asyncio
from datetime import timedelta

from temporalio import workflow

@workflow.defn(sandboxed=False)
class GenerateReportWorkflow:
    def __init__(self) -> None:
        self._current_prompt: str = ""
        self._user_decision: UserDecisionSignal = UserDecisionSignal(decision=UserDecision.WAIT)
        self._research_result: # TODO Initialize the _research_result to be an empty string

    # Method to handle the Signal
    @workflow.signal
    async def user_decision_signal(self, decision_data: UserDecisionSignal) -> None:
        # Update the instance variable with the received Signal data
        self._user_decision = decision_data
    
    @workflow.query # Query to get the current research result
    def get_research_result(self) -> str | None:
        # TODO: Return the `_research_result` field

    # Rest of code here...

In [ ]:
# Run this cell to load and display the solution
from pathlib import Path
from IPython.display import display, Markdown
import os

notebook_dir = Path(os.getcwd())
solution_file = notebook_dir / "Solutions_03_Human_in_the_Loop" / "handling_query_solution.py"

code = solution_file.read_text()

print("Solution loaded:")
display(Markdown(f"```python\n{code}\n```"))

### React to Query

We now want to fill out the contents of the research result so that the contents can be sent back to the user when the user queries for the research result.

In [ ]:
# Adding a Query into our Workflow code
# Step 1: After the LLM call, set `_research_result` field to be the contents of the research (research_facts["choices"][0]["message"]["content"])
# Step 2: Run this code block to load it into the program
import asyncio
from datetime import timedelta

from temporalio import workflow

@workflow.defn(sandboxed=False)
class GenerateReportWorkflow:
    def __init__(self) -> None:
        self._current_prompt: str = ""
        # Instance variable to store the Signal in
        self._user_decision: UserDecisionSignal = UserDecisionSignal(decision=UserDecision.WAIT)
        self._research_result: str = ""

    # Method to handle the Signal
    @workflow.signal
    async def user_decision_signal(self, decision_data: UserDecisionSignal) -> None:
        # Update the instance variable with the received Signal data
        self._user_decision = decision_data

    @workflow.query # Query to get the current research result
    def get_research_result(self) -> str | None:
        return self._research_result

    @workflow.run
    async def run(self, input: GenerateReportInput) -> GenerateReportOutput:
        self._current_prompt = input.prompt

        llm_call_input = LLMCallInput(
            prompt=self._current_prompt,
            llm_api_key=input.llm_api_key,
            llm_model=input.llm_research_model,
        )

        continue_agent_loop = True

        while continue_agent_loop:
            research_facts = await workflow.execute_activity(
                llm_call,
                llm_call_input,
                start_to_close_timeout=timedelta(seconds=30),
            )

            # TODO: Set `_research_result` field to be the contents of the research (research_facts["choices"][0]["message"]["content"])

            # Waiting for Signal with user decision
            await workflow.wait_condition(lambda: self._user_decision.decision != UserDecision.WAIT)

            if self._user_decision.decision == UserDecision.KEEP:
                workflow.logger.info("User approved the research. Creating PDF...")
                continue_agent_loop = False
            elif self._user_decision.decision == UserDecision.EDIT:
                workflow.logger.info("User requested research modification.")
                if self._user_decision.additional_prompt != "":
                    self._current_prompt = (
                        f"{self._current_prompt}\n\nAdditional instructions: {self._user_decision.additional_prompt}"
                    )
                    workflow.logger.info(f"Regenerating research with updated prompt: {self._current_prompt}")
                else:
                    workflow.logger.info("No additional instructions provided. Regenerating with original prompt.")
                llm_call_input.prompt = self._current_prompt

                self._user_decision = UserDecisionSignal(decision=UserDecision.WAIT)

        pdf_generation_input = PDFGenerationInput(content=research_facts["choices"][0]["message"]["content"])

        pdf_filename: str = await workflow.execute_activity(
            create_pdf,
            pdf_generation_input,
            start_to_close_timeout=timedelta(seconds=20),
        )

        return GenerateReportOutput(result=f"Successfully created research report PDF: {pdf_filename}")

In [ ]:
# Run this cell to load and display the solution
from pathlib import Path
from IPython.display import display, Markdown
import os

notebook_dir = Path(os.getcwd())
solution_file = notebook_dir / "Solutions_03_Human_in_the_Loop" / "react_to_query_solution.py"

code = solution_file.read_text()

print("Solution loaded:")
display(Markdown(f"```python\n{code}\n```"))

## Sending a Signal and Query with the Client

To send a Signal with the Temporal Client, we need to get a "handle" to a specific Workflow Execution, which will be used to interact with that Workflow.

We'll do this with the `get_workflow_handle` method.

```python
handle = client.get_workflow_handle(workflow_id)
```

## Sending a Query wiht the Client

Let's send a Query first. 

1. Get a handle of the Workflow Execution we will query
2. Send a query with the `query` method.

```python
research_result = await handle.query(GenerateReportWorkflow.get_research_result)
```

In [ ]:
# Let's send our Query from the Client code
# TODO: Run this codeblock

async def query_research_result(client: Client, workflow_id: str) -> None:
    handle = client.get_workflow_handle(workflow_id)

    try:
        research_result = await handle.query(GenerateReportWorkflow.get_research_result)
        if research_result:
            print(f"Research Result: {research_result}")
        else:
            print("Research Result: Not yet available")

    except Exception as e:
        print(f"Query failed: {e}")

In [ ]:
# Let's send our Signal from the Client code
# Step 1: If the input decision was to edit the the prompt, send a Signal to your Workflow Handle with the new signal data
# Step 2: Run this code block to load it into the program
async def send_user_decision_signal(client: Client, workflow_id: str) -> None:
  loop = asyncio.get_running_loop() # We usually do not need this
  handle = client.get_workflow_handle(workflow_id) # Get a handle on the Workflow Execution we want to send a Signal to.

  while True:
      print("\n" + "=" * 80)
      print(
          "Calling LLM! See the response in your Web UI in the output of the `llm_call` Activity. Would you like to keep or edit it?"
      )
      print("1. Type 'keep' to approve the output and create PDF")
      print("2. Type 'edit' to modify the output")
      print("=" * 80)

      # When running input in async code, run in an executor to not block the event loop
      decision = await loop.run_in_executor(None, input, "Your decision (keep/edit): ")
      decision = decision.strip().lower()

      if decision in {"keep", "1"}:
          signal_data = UserDecisionSignal(decision=UserDecision.KEEP)
          await handle.signal("user_decision_signal", signal_data) # Send our Keep Signal to our Workflow Execution we have a handle on
          print("Signal sent to keep output and create PDF")
          break
      if decision in {"edit", "2"}:
          additional_prompt_input = input("Enter additional instructions to edit the output (optional): ").strip()
          additional_prompt = additional_prompt_input if additional_prompt_input else ""
          signal_data = UserDecisionSignal(decision=UserDecision.EDIT, additional_prompt=additional_prompt)
          # TODO Send our Signal to our Workflow Execution we have a handle on
          print("Signal sent to regenerate output")

      else:
          print("Please enter either 'keep', 'edit'")")

In [ ]:
# Run this cell to load and display the solution
from pathlib import Path
from IPython.display import display, Markdown
import os

notebook_dir = Path(os.getcwd())
solution_file = notebook_dir / "Solutions_03_Human_in_the_Loop" / "sending_signal_solution.py"

code = solution_file.read_text()

print("Solution loaded:")
display(Markdown(f"```python\n{code}\n```"))

In [ ]:
# Starting the Worker again
worker = asyncio.create_task(run_worker())

 # Check if the task is in the set of all tasks
if worker in asyncio.all_tasks():
    # The sleep is necessary because of the async task scheduling in Jupyter
    print("Task is currently active.") # The Worker now registers the updated Workflow changes
else:
    print("Task is not found in active tasks (might have finished or not yet scheduled).")

In [ ]:
# Now send your Signal by running this code block.
send_signal = await send_user_decision_signal(client, handle.id)

## Putting it Together

You can now make a research call, check your results by querying your Workflow, then choose to edit or keep the research!

---
# Exercise 3 - Human in the Loop

* In these exercises you will:
  * Review a modified version of the previous exercise and investigate the results in the Web UI
  * Add a Signal to the exercise to provide the filename you wish to save the research report as
  * Add a Query to the exercise to extract the character length of the research request
* Go to the **Exercise** Directory in the Google Drive and open the **Practice** Directory
* Open _01-An-AI-Agent-Practice.ipynb_ and follow the instructions
* If you get stuck, raise your hand and someone will come by and help. You can also check the `Solution` directory for the answers
* **You have 5 mins**

## What's Next?

This workshop introduced you to the **interacting with workflows** in Temporal. Further your learning with these resources:

- Our [free course](https://learn.temporal.io/courses/interacting_with_workflows/python/) on Signals and Queries in Temporal and other ways to interact with Workflows
- A [Python tutorial](https://learn.temporal.io/tutorials/python/build-an-email-drip-campaign/) to practice handling Signals and Queries
- Documentation on [Updates](https://docs.temporal.io/develop/python/message-passing#updates), a feature which combine both Signals and Queries